In [1]:
# RSI PowerZones Weekly
import gta_indicator
import gta_prices
import gta_signals
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'IBPO'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
history_start = '2010-09-14'
p1 = 48 # System #1 look back period.
p2 = 24 # System #2 look back period.
p3 = 12 # System #3 look back period.
p4 = 6 # System #4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
charges = 11.95  # Commission per trade.
stamp_duty = 0.0  # Stamp Duty percentage.

In [3]:
def trades_source(prices, period):
    '''Trade entries & exits.'''
    dc = gta_indicator.donchian(prices, period)
    td = pd.concat([prices, dc], axis=1)
    td['buy'] = np.where(td.close > td.upr, 1, 0)
    td['sell'] = np.where(td.close < td.mid, 1, 0)
    td['state'] = gta_signals.state_signal(td.buy, td.sell, period)
    td['entry'] = np.where(np.logical_and(td.state == 1, td.state.shift(periods=1) == 0), 1, 0)
    td['exit'] = np.where(np.logical_and(td.state == 0, td.state.shift(periods=1) == 1), 1, 0)
    df = pd.concat([td.close, td.entry, td.exit], axis=1)
    return df

def trades_derived(prices, period, entry):
    '''Trade entries & exits.'''
    dc = gta_indicator.donchian(prices, period)
    td = pd.concat([prices, dc], axis=1)
    td['buy'] = entry
    td['sell'] = np.where(td.close < td.mid, 1, 0)
    td['state'] = gta_signals.state_signal(td.buy, td.sell, period)
    td['entry'] = np.where(np.logical_and(td.state == 1, td.state.shift(periods=1) == 0), 1, 0)
    td['exit'] = np.where(np.logical_and(td.state == 0, td.state.shift(periods=1) == 1), 1, 0)
    df = pd.concat([td.close, td.entry, td.exit], axis=1)
    return td

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm, history_start)

In [5]:
# Trade entries & exits.
td1 = trades_source(prices, p1)
td2 = trades_derived(prices, p2, td1['entry'])
td3 = trades_derived(prices, p3, td1.entry)
td4 = trades_derived(prices, p4, td1.entry)
td = pd.concat([td1, td2, td3, td4], axis=0)
td = td.sort_index()
td = td.drop_duplicates()

In [6]:
# Position size.
# td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)
# td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
# td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)

# Modify number of shares to be divisible by 4.
# for i in range(len(td)):
#     while td.loc[td.index[i], 'shares'] % 4 != 0:
#         td.loc[td.index[i], 'shares'] += 1

# Charges & stamp duty
# td['charges'] = charges
# td['stamp_duty'] = ((stamp_duty / 100) * td.close * td.shares).round(2)

# Cost
# td['cost'] = np.where(td.entry == 1, (td.close * td.shares) + td.charges + td.stamp_duty, 0)
td2

,open,high,low,close,upr,lwr,mid,buy,sell,state,entry,exit
date,,,,,,,,,,,,
2010-09-17,1.225,1.400,1.200,1.375,NaN,NaN,NaN,0,0,0,0,0
2010-09-24,1.375,1.375,1.350,1.375,NaN,NaN,NaN,0,0,0,0,0
2010-10-01,1.375,1.400,1.350,1.400,NaN,NaN,NaN,0,0,0,0,0
2010-10-08,1.395,1.540,1.395,1.540,NaN,NaN,NaN,0,0,0,0,0
2010-10-15,1.540,1.550,1.540,1.540,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,3.910,4.000,3.780,3.860,3.94,2.48,3.21,0,0,0,0,0
2022-04-01,3.980,4.000,3.840,3.860,4.00,2.54,3.27,0,0,0,0,0
2022-04-08,3.950,3.990,3.820,3.920,4.00,2.54,3.27,0,0,0,0,0
